In [ ]:
#this query will identify all cardiovascular deaths in the care hme cohort following admission to the care home in one year. this will be done based on the ICD 
#call all the deaths in the carehome cohort 
with deaths_carehome as(
select 
*
from yhcr-prd-bradfor-bia-core.CB_FDM_DeathCertificates.tbl_NEC_Deaths
where person_id in (select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` )),

#left join ch_admission date 
death_carehome_merge as (
  select
dch.person_id, tbl_NEC_Deaths_end_date,dch.dod,reg_date,s_cod_code_1,S_COD_LINE_1,s_cod_code_4,
  ch.first_episodestartdate
  from deaths_carehome dch
  left join yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1 ch
  on dch.person_id = ch.person_id
),

#got 3 duplicate ntries for a lot of people, there doesn't seem to be any difference between the duplicate excpet that some are missing the dod and reg date, therefore i have used a row_number function to select the duplicate htat contains a dod and reg_date 
dch_rank as (
select
*,
row_number() over (partition by person_id order by dod, reg_date desc) as duplicate_rank
 from 
death_carehome_merge),

duplicate_remove  as(
select
person_id, tbl_NEC_Deaths_end_date  as dod,first_episodestartdate
from dch_rank
where (duplicate_rank = 1) and ((s_cod_code_1 like 'I%') OR (s_cod_code_1 like '%irculat%') or (s_cod_code_4 like '%irculatory%')) and (tbl_NEC_Deaths_end_date > (EXTRACT(date from first_episodestartdate))))

select
*, case when person_id is not null then "cv death" end as event_type
from duplicate_remove 
where date_diff(duplicate_remove.dod, (EXTRACT(date from first_episodestartdate)),day) <366

